# Literary Analysis: Comparing Nonfiction and Fiction through Topic Modeling and Sentiment Analysis 

### ADS 509 Final Project
##### Team 3: Claire Bentzen, Tara Dehdari, Logan Van Dine

##### Introduction

In this project, we will conduct a comparative analysis of two significant literary works: "Pride and Prejudice" by Jane Austen (fiction) and "A Vindication of the Rights of Woman" by Mary Wollstonecraft (nonfiction). Both texts engage deeply with themes of gender, society, and individual rights, making them ideal for exploring the differences in language, themes, and sentiment between fiction and nonfiction.

Using text mining techniques, we will analyze how each genre approaches these themes, examining the stylistic and rhetorical differences that characterize fiction versus nonfiction. Our analysis will involve data cleaning, tokenization, and the application of descriptive statistics, sentiment analysis, and topic modeling. By comparing these works, we aim to uncover the unique ways in which each genre communicates similar ideas, providing insights into the broader distinctions between fiction and nonfiction writing.


### Imports

In [17]:
import requests
from bs4 import BeautifulSoup  
import os

### Scraping

This portion scrapes and saves the full text of Pride and Prejudice and A Vindication of the Rights of Woman from Project Gutenberg. It ensures the save directory exists, extracts text from the HTML, saves it to .txt files, and verifies that the files were created successfully, showing a preview of the content.

In [19]:
# Define the URLs for the books
url_pride_prej = 'https://www.gutenberg.org/cache/epub/1342/pg1342-images.html'
url_vin_of_women = 'https://www.gutenberg.org/cache/epub/3420/pg3420-images.html'

# Define the directory to save the files
data_dir = './data'

# Ensure the directory exists
os.makedirs(data_dir, exist_ok=True)

# Function to scrape and save books
def scrape_and_save_book(url, file_name):
    # Send a GET request to the URL
    response = requests.get(url)
    response.raise_for_status()  # Check that the request was successful
    
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract all text from <p> tags
    paragraphs = soup.find_all('p')
    book_text = '\n'.join([para.get_text() for para in paragraphs])
    
    # Save the extracted text to a file
    file_path = os.path.join(data_dir, file_name)
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(book_text)
    
    print(f"Text from '{file_name}' scraped and saved.")
    
    # Check if the file is saved and contains content
    if os.path.exists(file_path):
        print(f"File '{file_path}' has been created successfully.")
        # Check the first few lines of the file
        with open(file_path, 'r', encoding='utf-8') as file:
            preview = file.read(500)  # Read the first 500 characters
            print("File content preview:\n")
            print(preview)
    else:
        print(f"Failed to create the file '{file_path}'.")

# Scrape and save Pride and Prejudice
scrape_and_save_book(url_pride_prej, 'pride_and_prejudice.txt')

# Scrape and save A Vindication of the Rights of Woman
scrape_and_save_book(url_vin_of_women, 'vindication_of_rights_of_woman.txt')

Text from 'pride_and_prejudice.txt' scraped and saved.
File './data\pride_and_prejudice.txt' has been created successfully.
File content preview:

Title: Pride and Prejudice
Author: Jane Austen
Release date: June 1, 1998 [eBook #1342]

                Most recently updated: June 17, 2024
Language: English
Credits: Chuck Greif and the Online Distributed Proofreading Team at http://www.pgdp.net (This file was produced from images available at The Internet Archive)

PREFACE.
List of Illustrations.
Chapter: I., 
II., 
III., 
IV., 
V., 
VI., 
VII., 
VIII., 
IX., 
X., 
XI., 
XII., 
XIII., 
XIV., 
XV., 
XVI., 
XVII., 
XVIII., 
XIX., 
XX., 
XXI., 
Text from 'vindication_of_rights_of_woman.txt' scraped and saved.
File './data\vindication_of_rights_of_woman.txt' has been created successfully.
File content preview:

Title: A Vindication of the Rights of Woman
Author: Mary Wollstonecraft
Release date: September 1, 2002 [eBook #3420]

                Most recently updated: January 8, 2021
Language:

### Data Cleaning

### Data Tokenization